In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
matplotlib.rcParams['svg.fonttype'] = 'none' # for svgs
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_map.figure_utils import depth_selectivity, get_session_list, depth_decoder
from v1_depth_map.figure_utils import common_utils as plt_common_utils

In [ ]:
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
READ_VERSION = 10
VERSION = 10
READ_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_2023"/f"ver{READ_VERSION}"
SAVE_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_2023"/f"ver{VERSION}"
SAVE_ROOT.mkdir(parents=True, exist_ok=True)
(SAVE_ROOT/"fig1").mkdir(parents=True, exist_ok=True)

reload = False # set to true if you want to reload data

In [ ]:
# Load data
# Load an example session
project = "hey2_3d-vision_foodres_20220101"
session_name = "PZAH8.2h_S20230116"
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)
suite2p_ds = flz.get_datasets_recursively(
    flexilims_session=flexilims_session,
    origin_name=session_name,
    dataset_type="suite2p_traces",
)
fs = list(suite2p_ds.values())[0][-1].extra_attributes["fs"]

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)

suite2p_ds = flz.get_datasets(
    flexilims_session=flexilims_session,
    origin_name=session_name,
    dataset_type="suite2p_rois",
    filter_datasets={"anatomical_only": 3},
    allow_multiple=False,
    return_dataseries=False,
)
stat = np.load(suite2p_ds.path_full / "plane0" / "stat.npy", allow_pickle=True)
iscell = np.load(suite2p_ds.path_full / "plane0" / "iscell.npy", allow_pickle=True)[
    :, 0
]
ops = np.load(suite2p_ds.path_full / "plane0" / "ops.npy", allow_pickle=True).item()

In [ ]:
# Concatenate neurons_df from all closedloop sessions (including sessions with openloop)
mouse_list = flz.get_entities("mouse", flexilims_session=flexilims_session)
mouse_list = mouse_list[mouse_list.name.isin(["PZAH6.4b",
                "PZAG3.4f",
                "PZAH8.2h",
                "PZAH8.2i",
                "PZAH8.2f",
                "PZAH10.2d",
                "PZAH10.2f"])]
if reload:
    neurons_df_all = plt_common_utils.concatenate_all_neurons_df(
        flexilims_session=flexilims_session,
        session_list=get_session_list.get_sessions(
        flexilims_session,
        exclude_openloop=False,
        exclude_pure_closedloop=False,
        v1_only=True,
        trialnum_min=10,
        mouse_list=mouse_list,
    ),
        filename="neurons_df.pickle",
        cols=[
            "roi",
            "is_depth_neuron",
            "depth_neuron_anova_p",
            "best_depth",
            "preferred_depth_closedloop",
            "depth_tuning_popt_closedloop",
            "depth_tuning_trials_closedloop_crossval",
            "preferred_depth_closedloop_crossval",
            "depth_tuning_popt_closedloop_running",
            "preferred_depth_closedloop_running",
            "depth_tuning_popt_closedloop_notrunning",
            "preferred_depth_closedloop_notrunning",
            "depth_tuning_test_rsq_closedloop",
            "depth_tuning_test_spearmanr_rval_closedloop",
            "depth_tuning_test_spearmanr_pval_closedloop",
        ],
        read_iscell=True,
        verbose=True,
    )
    neurons_df_all.to_pickle(SAVE_ROOT / "fig1/neurons_df_all.pickle")
else:
    neurons_df_all = pd.read_pickle(READ_ROOT / "fig1/neurons_df_all.pickle")
neurons_df_all = neurons_df_all[neurons_df_all["iscell"] == 1]

# Load the depth tuning raster. To recompute, use make_depth_tuning_raster.py 
if reload:  
    results_all = depth_selectivity.get_psth_crossval_all_sessions(
        flexilims_session,
        session_list=get_session_list.get_sessions(
            flexilims_session,
            exclude_openloop=False,
            exclude_pure_closedloop=False,
            v1_only=True,
            trialnum_min=10,
            mouse_list=mouse_list,
        ),
        nbins=60,
        closed_loop=1,
        use_cols=[
            "roi",
            "is_depth_neuron",
            "depth_neuron_anova_p",
            "best_depth",
            "preferred_depth_closedloop",
            "depth_tuning_popt_closedloop",
            "depth_tuning_trials_closedloop",
            "depth_tuning_trials_closedloop_crossval",
            "preferred_depth_closedloop_crossval",
            "depth_tuning_test_rsq_closedloop",
            "depth_tuning_test_spearmanr_rval_closedloop",
            "depth_tuning_test_spearmanr_pval_closedloop",
        ],
        blank_length=3,
        overwrite=False,
    ) 
    results_all.to_pickle(SAVE_ROOT / "fig1/results_all_psth.pickle")
else:
    results_all = pd.read_pickle(READ_ROOT / "fig1/results_all_psth.pickle")

# print some stats
print(f"5 depth sessions: {len(results_all[results_all.session.str.contains('PZAH6.4b|PZAG3.4f')].session.unique())}")
print(f"8 depth sessions: {len(results_all[~results_all.session.str.contains('PZAH6.4b|PZAG3.4f')].session.unique())}")
print(f"Total depth sessions: {len(neurons_df_all.session.unique())}")
print(f"All neurons: {len(neurons_df_all)}")
print(f"Depth neurons: {((neurons_df_all['depth_tuning_test_spearmanr_rval_closedloop'] > 0.1) & (neurons_df_all['depth_tuning_test_spearmanr_pval_closedloop'] < 0.05)).sum()}")
print(f"Mean proportion of depth neurons {((neurons_df_all['depth_tuning_test_spearmanr_rval_closedloop'] > 0.1) & (neurons_df_all['depth_tuning_test_spearmanr_pval_closedloop'] < 0.05)).mean()}")


In [ ]:
# Load depth decoder results from all closedloop sessions (including sessions with openloop)
mouse_list = flz.get_entities("mouse", flexilims_session=flexilims_session)
mouse_list = mouse_list[mouse_list.name.isin(["PZAH6.4b",
                "PZAG3.4f",
                "PZAH8.2h",
                "PZAH8.2i",
                "PZAH8.2f",
                "PZAH10.2d",
                "PZAH10.2f"])]
if reload:
    decoder_df = plt_common_utils.concatenate_all_neurons_df(
        flexilims_session,
        session_list=get_session_list.get_sessions(
            flexilims_session,
            exclude_openloop=False,
            exclude_pure_closedloop=False,
            v1_only=True,
            trialnum_min=10,
            mouse_list=mouse_list,
        ),
        filename="decoder_results.pickle",
        cols=["accuracy_closedloop", "conmat_closedloop", "best_C_closedloop", "acc_speed_bins_closedloop", "conmat_speed_bins_closedloop"],
        read_iscell=False,
        verbose=True,
    )

    decoder_df.to_pickle(SAVE_ROOT / "decoder_df_all.pickle")
else:
    decoder_df = pd.read_pickle(READ_ROOT / "decoder_df_all.pickle")
print(f"Total depth decoder sessions: {len(decoder_df.session.unique())}") # should be the same as the session number above

In [ ]:
# Plot Fig.1
fontsize_dict = {"title": 7, "label": 7, "tick": 5, "legend": 5}
cm = 1 / 2.54
# Raster plot of an example neuron
EXAMPLE_ROI = 250
fig = plt.figure(figsize=(18 * cm, 18 * cm))
_, ax = depth_selectivity.plot_raster_all_depths(
    trials_df=trials_df_example,
    roi=EXAMPLE_ROI,
    is_closed_loop=True,
    corridor_length=6,
    blank_length=3,
    nbins=60,
    vmax=3,
    plot=True,
    cbar_width=0.01,
    fontsize_dict=fontsize_dict,
    position=[0.05, 0.75, 0.3, 0.15],
)
plt_common_utils.draw_axis_scalebars(ax=fig.add_axes([0, 0, 1, 1]), 
                                     scalebar_x=0.059, 
                                     scalebar_y=0.91, 
                                     scalebar_width=0.02, 
                                     scalebar_height=0.05, 
                                     scalebar_labels=["6 m of travel", ""], 
                                     xlim=None, 
                                     ylim=None, 
                                     label_fontsize=5, 
                                     linewidth=2, 
                                     right=False, 
                                     bottom=True)

# Example running & PSTH of a single neuron
ax = fig.add_axes([0.51, 0.81, 0.18, 0.1])
depth_selectivity.plot_PSTH(
    trials_df=trials_df_example,
    roi=EXAMPLE_ROI,
    is_closed_loop=True,
    use_col="dff",
    corridor_length=6,
    blank_length=3,
    nbins=60,
    frame_rate=fs,
    fontsize_dict=fontsize_dict,
    linewidth=1,
    legend_on=True,
    show_ci=False,
    ylim=(None, 1.3),
)
ax.set_yticks([0, 1.3])
ax.set_xticks([])
ax.set_xlabel("")

ax = fig.add_axes([0.51, 0.75, 0.18, 0.05])
depth_selectivity.plot_PSTH(
    trials_df=trials_df_example,
    roi=EXAMPLE_ROI,
    is_closed_loop=True,
    use_col="RS",
    corridor_length=6,
    blank_length=3,
    nbins=60,
    frame_rate=fs,
    fontsize_dict=fontsize_dict,
    linewidth=1,
    legend_on=False,
    show_ci=False,
    ylim=(0,115),
)
ax.set_ylabel("Running \nspeed (cm/s)", fontsize=fontsize_dict["label"])

# Example depth tuning curve & PSTH of a single neuron
ax=fig.add_axes([0.8, 0.75, 0.15, 0.15])
depth_tuning_kwargs = dict(
    rs_thr=None,
    plot_fit=True,
    plot_smooth=False,
    linewidth=2,
    linecolor="royalblue",
    closed_loop=1,
    fontsize_dict=fontsize_dict,
    markersize=10,
    markeredgecolor='w',
)
depth_selectivity.plot_running_stationary_depth_tuning(roi=EXAMPLE_ROI, roi_num=1, i=0, ax=ax, 
                                                           neurons_df=neurons_df_example,
                                                           trials_df=trials_df_example,
                                     depth_tuning_kwargs=depth_tuning_kwargs, 
                                     fontsize_dict=fontsize_dict, legend_loc="upper left")
ax.set_ylabel("\u0394F/F", fontsize=fontsize_dict["label"])
ax.set_xlabel(f"Virtual depth (cm)", fontsize=fontsize_dict["label"])

# Example depth tuning curve of 6 other neuronsn (running vs not running)
EXAMPLE_ROIS = [174, 249, 319, 638, 333, 696]

neurons_df_example["depth_tuned"] = neurons_df_example.apply(
    lambda x: x["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1
    and x["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05,
    axis=1,
)

overlay = np.zeros((ops["meanImg"].shape[0], ops["meanImg"].shape[1], 4))
for i, roi in enumerate(stat):
    if iscell[i]:
        overlay[roi["ypix"], roi["xpix"]] = [0, 1, 1, 0.25]
    # if i in EXAMPLE_ROIS or i == EXAMPLE_ROI:
    #     overlay[roi["ypix"], roi["xpix"]] = [1, 0, 0, 0.75]
    if neurons_df_example["depth_tuned"].iloc[i]:
        overlay[roi["ypix"], roi["xpix"]] = [1, 0, 0, 0.75]
        
fov_ax = fig.add_axes([0.35, 0.25, 0.48, 0.48])
depth_selectivity.plot_fov_mean_img(ops["meanImg"])
plt.imshow(np.fliplr(overlay))
fov_ax.annotate(
    "1",
    (
        ops["meanImg"].shape[0] - stat[EXAMPLE_ROI]["med"][1],
        stat[EXAMPLE_ROI]["med"][0],
    ),
    xytext=(5, 5),
    textcoords="offset points",
    color="w",
    fontsize=fontsize_dict["label"],
    arrowprops=dict(facecolor="w", edgecolor="w", arrowstyle="->"),
)
from matplotlib.lines import Line2D

legend_elements = [
    Line2D(
        [0],
        [0],
        marker="o",
        markersize=3,
        linestyle="none",
        color="cyan",
        label="All neurons",
    ),
    Line2D(
        [0],
        [0],
        marker="o",
        markersize=3,
        linestyle="none",
        color="red",
        label="Depth-tuned neurons",
    ),
]
fov_ax.legend(
    handles=legend_elements,
    loc="upper right",
    fontsize=fontsize_dict["label"],
    frameon=True,
    framealpha=0.8,
    borderpad=0.6,
)

depth_tuning_kwargs = dict(
    rs_thr=None,
    plot_fit=True,
    plot_smooth=False,
    linewidth=1.5,
    linecolor="royalblue",
    closed_loop=1,
    fontsize_dict=fontsize_dict,
    markersize=8,
    markeredgecolor='w',
)
for i, roi in enumerate(EXAMPLE_ROIS):
    ax = fig.add_axes([i // 3 * 0.16+0.06, 0.57 - i % 3 * 0.13, 0.11, 0.1])
    if i<3:
        label_pos="upper_right"
    else:
        label_pos="upper_left"
    
    roi_num = i+2
    depth_selectivity.plot_running_stationary_depth_tuning(roi=roi, roi_num=roi_num, i=i, 
                                                           neurons_df=neurons_df_example,
                                                           trials_df=trials_df_example,
                                                           ax=ax, fov_ax=fov_ax, ops=ops, stat=stat,
                                        depth_tuning_kwargs=depth_tuning_kwargs, 
                                        fontsize_dict=fontsize_dict, legend_loc="upper right", text_pos=label_pos)
    
# Histogram of proportion of depth-tuned neurons;
# preferred depths of all depth-tuned neurons;
neurons_df_all["depth_tuned"] = neurons_df_all.apply(
    lambda x: x["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1
    and x["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05,
    axis=1,
)

fig.add_axes([0.8, 0.54, 0.16, 0.13])
depth_selectivity.plot_depth_neuron_perc_hist(
    results_df=neurons_df_all,
    bins=np.arange(0, 1.0, 0.1),
    ylim=(0, 25),
    fontsize_dict=fontsize_dict,
)

# Histogram of preferred depths of all depth-tuned neurons;
fig.add_axes([0.8, 0.31, 0.16, 0.13])
depth_selectivity.plot_preferred_depth_hist(
    results_df=neurons_df_all[(neurons_df_all["depth_tuned"] == 1)],
    use_col="preferred_depth_closedloop",
    nbins=20,
    fontsize_dict=fontsize_dict,
)

# Raster plot of preferred depths of all neurons (sorted by preferred depth)
results_all["ndepths"] = results_all["psth_crossval"].apply(lambda x: len(x) - 1)
neurons_resp = results_all[
    (results_all["ndepths"] == 8)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (results_all["iscell"] == 1)
]
fig.add_axes([0.055, 0.06, 0.2, 0.15])
depth_selectivity.plot_psth_raster(
    results_df=neurons_resp,
    depth_list=np.round(np.geomspace(5, 640, num=8)).astype(int),
    fontsize_dict=fontsize_dict,
)
plt_common_utils.draw_axis_scalebars(ax=fig.add_axes([0, 0, 1, 1]), 
                                     scalebar_x=0.058, 
                                     scalebar_y=0.215, 
                                     scalebar_width=0.2*0.09, 
                                     scalebar_height=0.05, 
                                     scalebar_labels=["6 m of travel", ""], 
                                     xlim=None, 
                                     ylim=None, 
                                     label_fontsize=5, 
                                     linewidth=2, 
                                     right=False, 
                                     bottom=True)

# Plot decoder error for different running speed bins
decoder_df = depth_decoder.calculate_error_all_sessions(decoder_df)
ax1=fig.add_axes([0.39, 0.06, 0.03, 0.15])
ax2=fig.add_axes([0.43, 0.06, 0.14, 0.15])
_ = depth_decoder.plot_decoder_err_by_speeds(decoder_df, 
                            np.insert(np.linspace(0.2, 2, 10),0,0.05), 
                           axes=[ax1, ax2],
                           closed_loop=1, 
                           linecolor="k", 
                           linecolor_chance="k",
                           linewidth=1, 
                           linewidth_chance=0.5,
                           alpha=1,
                           alpha_chance=1,
                           markersize=5, 
                           fontsize_dict=fontsize_dict)
ax1.set_ylim([0,3])
ax2.set_ylim([0,3])

# Plot confusion  matrix for stationary & best running speed bin
ndepths=8
decoder_df = decoder_df[(decoder_df.apply(lambda x: len(x['conmat_closedloop']) == ndepths, axis=1))]
print(f"8 depth decoder sessions {len(decoder_df.session.unique())}")
conmats = []
for ibin in [0,3]:
    conmat_mean = depth_decoder.calculate_average_confusion_matrix(
        decoder_df,
        col=f'conmat_speed_bins_closedloop_{ibin}', 
        recording_types=["closedloop"],
        )
    conmat_closed = conmat_mean["closedloop"]
    conmat_closed = conmat_closed / conmat_closed.sum(axis=1)[:, np.newaxis]
    if ibin == 3:
        vmax = conmat_closed.max()
    conmats.append(conmat_closed)
vmax = np.round(vmax, 1)

if len(conmat_closed) == 8:
    depths = np.logspace(np.log2(5), np.log2(640), 8, base=2)
else:
    depths = np.logspace(np.log2(6), np.log2(600), 5, base=2)
depths = np.round(depths).astype(int)
    
for iplot, (title, colorbar_on) in enumerate(zip(["Stationary", "Running speed 40 - 60 cm/s"], [0,1])):
    ax = fig.add_axes([0.655 + 0.17 * iplot, 0.06, 0.13, 0.13])
    depth_decoder.plot_confusion_matrix(conmats[iplot], ax, vmax, fontsize_dict, depths=depths, 
                                        colorbar_on=colorbar_on,
                                        xtick_rotation=45,
                                        )
    ax.set_title(title, fontsize=fontsize_dict["title"])
    if iplot==1:
        ax.set_xlabel("")
        ax.set_ylabel("")


fig.savefig(SAVE_ROOT / "fig1.svg", bbox_inches="tight", dpi=300)